# Graph Building & Matching

In [ ]:
%load_ext autoreload
%autoreload 2

# %matplotlib widget
# above line is for interactive figures (nice for 3D plots). Install with:
# conda install -c conda-forge ipympl
import numpy as np
import matplotlib.pyplot as plt
import random
import quaternion as qt
#from scipy.spatial.transform import Rotation as R

import habitat_sim
from habitat_sim import AgentConfiguration, CameraSensorSpec

from tbp.monty.simulators.habitat import HabitatSim
from tbp.monty.frameworks.environments.habitat import PanTiltZoomCamera
from tbp.monty.frameworks.environment_utils.transforms import DepthTo3DLocations

from graph_utils import *

import warnings
warnings.filterwarnings('ignore')

## Habitat Experiment Setup

In [ ]:
# def get_one_obs_sim(obj="mug",obj_position=(0.01, 1.5, -0.08),cam_position=(0.0,0.0,0.0),cam_rotation=(1.0,0.0,0.0,0.0), obj_rotation=(0.0,0.0,0.0,0.0), world_coord=True):
    
#     camera = PanTiltZoomCamera(semantic=True, resolution=(32, 32),position=cam_position, rotation=cam_rotation)
#     sim = HabitatSim(agents={"agent_01": camera.get_spec()})
#     obj = sim.add_object(name=obj, position=obj_position,rotation=obj_rotation)

#     state = sim.get_agent("agent_01").get_state()
#     states = sim.get_states()
#     print(state)
#     print(states)

#     obs = sim.get_observations()
#     # add 3D coordinates
#     transform = DepthTo3DLocations(module_id='agent_01', resolution=obs['agent_01']['depth'].shape, world_coord=world_coord)
#     obs = transform(obs, state=states)

#     sim.close()
#     return obs, None, state


### Get Observations for Different Perspectives

In [ ]:
obs, state = get_one_obs_sim(obj="cup", resolution=(1024, 1024), 
                             cam_position=(0.0, 0.1, -0.01),
                             cam_rotation=(1.0, -0.4, 0.0, 0.0), 
                             world_coord=True)

In [ ]:
plt.figure()
plt.subplot(2,3,1)
plt.imshow(obs['agent_01']['rgba'])
plt.title('RGBA')
plt.axis('off')
plt.subplot(2,3,2)
plt.imshow(obs['agent_01']['depth'])
plt.title('Depth')
plt.axis('off')
plt.subplot(2,3,3)
plt.imshow(obs['agent_01']['semantic'])
plt.title('Semantic')
plt.axis('off')

In [ ]:
# xy_range = obs['agent_01']['rgba'].shape[:2]
# xy = np.round(np.random.normal(xy_range[0]//6, xy_range[0]//6, 2))
# xy

### Saccade on Environment Observation (will be moved into Habitat)

In [ ]:
def fake_saccade(full_obs, x, y):#gaussian=True):
#    xy_range = full_obs['agent_01']['rgba'].shape[:2]
    
#     if gaussian:
#         xy = np.round(np.random.normal(xy_range[0]//2, xy_range[0]//1.4, 2)).astype(int)#random.randint(0,xy_range[0]-1),random.randint(0,xy_range[1]-1)
#         xy[xy>45] = 45
#         xy[xy<30] = 30
#         x,y = xy
#     else:
#         x,y = random.randint(0,xy_range[0]-1),random.randint(0,xy_range[1]-1)
    
    z = full_obs['agent_01']['depth'][y,x]
    obj = full_obs['agent_01']['semantic'][y,x]>0
    feat = full_obs['agent_01']['rgba'][y,x]
    return x, y, z, obj, feat

In [ ]:
def collect_fake_saccades(full_obs, xs, ys):#, gaussian=True):
    all_x, all_y, all_z, all_obj, all_feat = [],[],[],[],[]
    
    for n in range(len(xs)):
        x, y = xs[n], ys[n]
        
        x, y, z, obj, feat = fake_saccade(full_obs, x, y)#, gaussian=gaussian)
        all_x.append(x), all_y.append(y), all_z.append(z), all_obj.append(obj), all_feat.append(feat)
        
    return all_x, all_y, all_z, all_obj, all_feat

In [ ]:
xs1, ys1, zs1, objs1, feats = collect_fake_saccades(
    obs, 
    xs=[500, 510, 520, 530], 
    ys=[640, 640, 630, 610]
)

# Important: When indexing array, x and y are switched!

plt.figure()
plt.imshow(obs['agent_01']['depth'])
plt.xlim([0,1024])
plt.ylim([1024,0])
plt.plot(xs1, ys1, marker='o',color='lightblue', markerfacecolor='green', markersize=0)
plt.scatter(xs1, ys1, marker='o',c=np.array([objs1])*1,cmap='Reds',s=70)
plt.title("Gaussian")

plt.show()

In [ ]:
xs2, ys2, zs2, objs2, feats = collect_fake_saccades(
    obs, 
    xs=[480, 490, 500, 510], 
    ys=[620, 620, 610, 590]
)

plt.figure()
plt.imshow(obs['agent_01']['depth'])
plt.xlim([0,1024])
plt.ylim([1024,0])
plt.plot(xs2, ys2, marker='o',color='lightblue', markerfacecolor='green', markersize=0)
plt.scatter(xs2, ys2, marker='o',c=np.array([objs2])*1,cmap='Reds',s=70)
plt.title("Gaussian")

plt.show()

In [ ]:
xs3, ys3, zs3, objs3, feats = collect_fake_saccades(
    obs, 
    xs=[31 * 16, 33 * 16, 34 * 16, 33 * 16], 
    ys=[41 * 16, 41 * 16, 39 * 16, 39 * 16]
)

plt.figure()
plt.imshow(obs['agent_01']['depth'])
plt.xlim([0,1024])
plt.ylim([1024,0])
plt.plot(xs3, ys3, marker='o',color='lightblue', markerfacecolor='green', markersize=0)
plt.scatter(xs3, ys3, marker='o',c=np.array([objs3])*1,cmap='Reds',s=70)
plt.title("Gaussian")

plt.show()

In [ ]:
xs4, ys4, zs4, objs4, feats = collect_fake_saccades(
    obs, 
    xs=[30 * 16, 32 * 16, 34 * 16, 32 * 16], 
    ys=[39 * 16, 41 * 16, 38 * 16, 37 * 16]
)

plt.figure()
plt.imshow(obs['agent_01']['depth'])
plt.xlim([0,1024])
plt.ylim([1024,0])
plt.plot(xs4, ys4, marker='o',color='lightblue', markerfacecolor='green', markersize=0)
plt.scatter(xs4, ys4, marker='o',c=np.array([objs4])*1,cmap='Reds',s=70)
plt.title("Gaussian")

plt.show()

In [ ]:
def encode_displacements(xs, ys, zs, num_padding, size):
    sdr1 = np.zeros(size)
    sdr2 = np.zeros(size)
    
    vec1 = np.array([xs[1] - xs[0],
                ys[1] - ys[0],
                zs[1] - zs[0]])

    vec2 = np.array([xs[2] - xs[1],
                    ys[2] - ys[1],
                    zs[2] - zs[1]])

    vec3 = np.array([xs[3] - xs[2],
                    ys[3] - ys[2],
                    zs[3] - zs[2]])
    
    e = np.cross(vec1, vec2) / np.linalg.norm(np.cross(vec1, vec2))
    theta = np.arccos(np.dot(e, vec3) / np.linalg.norm(vec3))


    f = vec3 - (np.dot(vec3, e) * e)
    phi = np.arccos(np.dot(vec1, f) / (np.linalg.norm(vec1) * np.linalg.norm(f)))
    
    
    print(theta, phi)
    
    theta_ind = np.digitize(theta, np.linspace(0, 360, size))
    phi_ind = np.digitize(phi, np.linspace(0, 360, size))
    
    for i in range(theta_ind - num_padding, theta_ind + num_padding + 1):
        sdr1[i] = 1
        
    for i in range(phi_ind - num_padding, phi_ind + num_padding + 1):
        sdr2[i] = 1
        
    return np.concatenate([sdr1, sdr2])

In [ ]:
np.nonzero(encode_displacements(xs1, ys1, zs1, num_padding=0, size=1000))

In [ ]:
np.nonzero(encode_displacements(xs2, ys2, zs2, num_padding=0, size=1000))

In [ ]:
np.nonzero(encode_displacements(xs3, ys3, zs3, num_padding=0))

In [ ]:
np.nonzero(encode_displacements(xs4, ys4, zs4, num_padding=0))